<a href="https://colab.research.google.com/github/Madhurika1292/ANALYSIS-AND-PREDICTION-ON-RETURNS-ON-AGRICULTURE-INVESTMENT/blob/master/MRS_test_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install surprise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import surprise
import seaborn as sns
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from surprise import SVD
from textblob import TextBlob
from sklearn.metrics import mean_squared_error



import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.warn("deprecated", DeprecationWarning)

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Bidirectional, LSTM, BatchNormalization, Dropout
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Dense
import random

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [ ]:
# Importing dataset from drive
from google.colab import drive
drive.mount('/content/gdrive/')

import sys
sys.path.append('/content/gdrive/My Drive/Drugscom')

In [ ]:
#Loading data
Drugscom_train=pd.read_csv('/content/gdrive/My Drive/Drugscom/Drugscom_train.csv')
Drugscom_test=pd.read_csv('/content/gdrive/My Drive/Drugscom/Drugscom_test.csv')

In [ ]:
# Dropping null values
Drugscom_train = Drugscom_train.dropna(axis=0)
Drugscom_test = Drugscom_test.dropna(axis=0)

In [ ]:
#Changing the "Unnamed: 0" column to uniqueId as it represents the unique id of the drugs
Drugscom_train=Drugscom_train.drop('Unnamed: 0', axis='columns')
Drugscom_test=Drugscom_test.drop('Unnamed: 0', axis='columns')

In [ ]:
Drugscom_train.head()

In [ ]:
#Drugscom_train=Drugscom_train.drop(Drugscom_train[(Drugscom_train['rating'] > 4.0) & (Drugscom_train['rating'] < 6.0)].index)
#Drugscom_test=Drugscom_test.drop(Drugscom_test[(Drugscom_test['rating'] > 4.0) & (Drugscom_test['rating'] < 6.0)].index)

In [ ]:
#Drugscom_train['sentiment'] = np.where(Drugscom_train['rating'] >= 7, 1, 0)
#Drugscom_test['sentiment'] = np.where(Drugscom_test['rating'] >= 7, 1, 0)

In [ ]:
def specific_condition_data(df,condition):
  
  return df[df['condition']==condition]

In [ ]:
training_data=specific_condition_data(Drugscom_train,'Anxiety')
testing_data=specific_condition_data(Drugscom_test,'Anxiety')

In [ ]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

In [ ]:
training_data['polarity'] = training_data["clean_review"].apply(pol)
training_data['subjectivity'] = training_data["clean_review"].apply(sub)
testing_data['polarity'] = testing_data["clean_review"].apply(pol)
testing_data['subjectivity'] = testing_data["clean_review"].apply(sub)


In [ ]:
training_data.head()

In [ ]:
def sentiments(polarity):
 if (polarity >= 0) :
   return 1
 elif polarity < 0:
   return 0

In [ ]:
# Add sentiments to the data
training_data["sentiment"] = training_data["polarity"].apply(sentiments)
testing_data["sentiment"] = testing_data["polarity"].apply(sentiments)

In [ ]:
X_train=training_data.clean_review
y_train=training_data.sentiment
X_test=testing_data.clean_review
y_test=testing_data.sentiment

In [ ]:
cv = CountVectorizer(max_features = 8000, ngram_range = (4, 4))

In [ ]:
X_train_cv=cv.fit_transform(X_train)
X_test_cv=cv.transform(X_test)

In [ ]:
X_train_cv.shape

In [ ]:
def grid_best_parameter(X_train_cv,y_train,X_test_cv,y_test):

  #Report to carry out the metrices and scores of different models
  report_table = [[]]
  
  #creating parameter

  #C= [1.0, 0.5, 0.1]
  C=np.logspace(-50, 50, 100)
  alpha= [0.01, 0.1, 0.5, 1.0, 10.0]
  k_range = list(range(1, 16))
  
  

  # creating pipeline
  pipe_lr=Pipeline([('clf' , LogisticRegression(random_state=42))])
  pipe_mnb=Pipeline([('clf', MultinomialNB())])
  pipe_bnb=Pipeline([('clf', BernoulliNB())])
  pipe_svc = Pipeline([('clf', LinearSVC())])
  pipe_dtc = Pipeline([('clf', DecisionTreeClassifier())])
  pipe_rfc = Pipeline([('clf', RandomForestClassifier(random_state=42))])




  #creating parameter

  param_lr =[{'clf__penalty': ['l1', 'l2'],
		          'clf__C': C,
		          'clf__solver': ['liblinear','lbfgs']}] 

  param_mnb=[{'clf__alpha': alpha}]

  param_bnb=[{'clf__alpha': alpha}]
  param_svc = {'clf__C':np.arange(0.01,100,10)}
  param_dtc = {'clf__criterion':['gini','entropy'],'clf__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
  param_rfc = { 
    'clf__n_estimators': [200, 500],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__max_depth' : [4,5,6,7,8],
    'clf__criterion' :['gini', 'entropy']
}





  # creating gridsearch

  grid_lr=GridSearchCV(estimator=pipe_lr,param_grid=param_lr,scoring='accuracy',cv=10,n_jobs=-1)
  grid_mnb=GridSearchCV(estimator=pipe_mnb,param_grid=param_mnb,cv=10,verbose=True, n_jobs=-1)
  grid_bnb=GridSearchCV(estimator=pipe_bnb,param_grid=param_bnb,cv=10,verbose=True, n_jobs=-1)
  grid_svc = GridSearchCV(estimator=pipe_svc,param_grid=param_svc,cv=10,return_train_score=True, n_jobs=1)
  grid_dtc = GridSearchCV(estimator=pipe_dtc,param_grid=param_dtc,cv=10, n_jobs=1)
  grid_rfc = GridSearchCV(estimator=pipe_rfc,param_grid=param_rfc,cv=10, n_jobs=1)




  #listing pipelines

  grid=[grid_lr, grid_mnb, grid_bnb, grid_svc,grid_dtc, grid_rfc]

  #dictionary of pipelines and models

  grid_dict={0: 'Logistic Regression', 1: 'Multi Nomial NB', 2: 'Bernoulli NB', 3:'Linear SVC', 4:'DecisionTreeClassifier', 5:'RandomForestClassifier'}

  # fitting grid search 

  print('Performing model optimizations...')
  best_accuracy = 0.0
  best_model = 0
  best_grid = ''

  for i, g in enumerate(grid):
    print('\nEstimator: %s' % grid_dict[i])	
    # Fit grid search	
    g.fit(X_train_cv, y_train)
    # Best params
    #print('Best params: %s' % g.best_params_)
    # Best training data accuracy
    #print('Best training accuracy: %.3f' % g.best_score_)
    # Predict on train data with best params
    train_prediction = g.predict(X_train_cv)
    # Predict on test data with best params
    prediction = g.predict(X_test_cv)
    # Test data accuracy of model with best params
    #print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, prediction))
    try:
      #calculate roc_auc_score for training data
      roc_auc_score_train=roc_auc_score(train_prediction, y_train)

      #calculate roc_auc_score for testing data
      roc_auc_score_test=roc_auc_score(prediction, y_test)

      #calculate RMSE, MSE
      from math import sqrt
      rmse_score_test= sqrt(mean_squared_error(y_test, prediction))
      mse_score_test= mean_squared_error(y_test,prediction)


      #calculate f1_score,precision,recall
      from sklearn.metrics import precision_score, f1_score, recall_score


    except ValueError:
      pass
   
    # Track best (highest test accuracy) model
    report_table = report_table+ [[grid_dict[i], g.best_params_, g.score(X_train_cv, y_train), g.score(X_test_cv, y_test), roc_auc_score_train, roc_auc_score_test,rmse_score_test,mse_score_test,precision_score(y_test,prediction),f1_score(y_test,prediction),recall_score(y_test,prediction)]]  
    if accuracy_score(y_test, prediction) > best_accuracy:
      best_accuracy = accuracy_score(y_test, prediction)
      best_model = g
      best_grid = i
  #print('\nClassifier with best test set accuracy: %s' % grid_dict[best_grid])
  #print('\nAccuracy score:%s' % best_accuracy)
  report_table.pop(0)
  report = pd.DataFrame(report_table,columns = ['Model name', 'Model parameter', 'Train accuracy', 'Test accuracy', 'Train auc score', 'Test auc score','RMSE score','MSE score','Precision','F1_score','Recall'])
  return report


#https://www.kdnuggets.com/2018/01/managing-machine-learning-workflows-scikit-learn-pipelines-part-3.html

In [ ]:
report=grid_best_parameter(X_train_cv,y_train,X_test_cv,y_test)

In [ ]:
report

#references
https://www.kaggle.com/sociopath00/random-forest-using-gridsearchcv